In [4]:
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.indicators.gd import GD
from pymoo.indicators.igd import IGD
import matplotlib.pyplot as plt
import numpy as np
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination import get_termination
from random import randint
from pymoo.termination.ftol import SingleObjectiveSpaceTermination
from pymoo.termination.robust import RobustTermination
from pymoo.termination.default import DefaultMultiObjectiveTermination


In [2]:
class tester():
    algorithm1_win_execution_time = 0
    algorithm1_win_convergence = 0
    algorithm2_win_execution_time = 0
    algorithm2_win_convergence = 0
    
    def __init__(self, algo1, algo2, problem):
        self.algo1 = algo1
        self.algo2 = algo2
        self.problem = problem
        self.res1_list = []
        self.res2_list = []
        self.res1_exec_time_list = []
        self.res2_exec_time_list = []
        self.res1_gd_list = []
        self.res2_gd_list = []

    def execute(self):
        pf = self.problem.pareto_front()
        ind = GD(pf)

        for i in range(1,11):
            res1 = minimize(self.problem,
                self.algo1,
                termination = DefaultMultiObjectiveTermination(xtol=1e-32,
                                                               cvtol=1e-32,
                                                               ftol=1e-6,
                                                               period=20,
                                                               n_max_gen=300000,
                                                               n_max_evals=100000
                                                            ),
                save_history=True,
                seed = i,
                verbose=False)
            
            res2 = minimize(self.problem,
                self.algo2,
                termination = DefaultMultiObjectiveTermination(xtol=1e-32,
                                                               cvtol=1e-32,
                                                               ftol=1e-6,
                                                               period=20,
                                                               n_max_gen=300000,
                                                               n_max_evals=100000
                                                            ),
                save_history=True,
                seed = i,
                verbose=False)
            
            self.res1_list.append(res1)
            self.res2_list.append(res2)
            self.res1_exec_time_list.append(res1.exec_time)
            self.res2_exec_time_list.append(res2.exec_time)

            res1_b = minimize(self.problem,
                self.algo1,
                termination = get_termination("n_eval", 50000),
                save_history=True,
                seed = i,
                verbose=False)
            
            res2_b = minimize(self.problem,
                self.algo2,
                termination = get_termination("n_eval", 50000),
                save_history=True,
                seed = i,

                verbose=False)
            if res1_b.F is None:
                self.res1_gd_list.append(None)
            else:
                self.res1_gd_list.append(ind(res1_b.F))
            if res2_b.F is None:
                self.res2_gd_list.append(None)
            else:
                self.res2_gd_list.append(ind(res2_b.F))
        
        #print("Best solution found by algorithm 1: \nX = %s\nOptimum found = %s\nCV = %s" % (res1.X, res1.F, res1.CV))
        #print("Best solution found by algorithm 2: \nX = %s\nOptimum found = %s\nCV = %s \n" % (res2.X, res2.F, res2.CV))
        
        algo1_avg_execute_time = sum(self.res1_exec_time_list) / len(self.res1_exec_time_list)
        algo2_avg_execute_time = sum(self.res2_exec_time_list) / len(self.res2_exec_time_list)
        algo1_execute_time_std = np.std(np.array(self.res1_exec_time_list))
        algo2_execute_time_std = np.std(np.array(self.res2_exec_time_list))
        
        
       
        
       
        
        print('Average execution time of algorithm 1:', algo1_avg_execute_time)
        print('Average execution time of algorithm 2:', algo2_avg_execute_time,'\n')

        print('Shortest execution time of algorithm 1:', min(self.res1_exec_time_list))
        print('Shortest execution time of algorithm 2:', min(self.res2_exec_time_list),'\n')
        
        print('Standard deviation of execution time of algorithm 1:', algo1_execute_time_std)
        print('Standard deviation of execution time of algorithm 2:', algo2_execute_time_std,'\n')
        
        if algo1_avg_execute_time < algo2_avg_execute_time:
            tester.algorithm1_win_execution_time += 1
            print("Algorithm 1 has shorter average execution time\n")
        else:
            tester.algorithm2_win_execution_time += 1
            print("Algorithm 2 has shorter average execution time\n")

        if None in self.res1_gd_list:
            print("No feasible solution found by algorithm 1")
        else:
            algo1_avg_gd = sum(self.res1_gd_list) / len(self.res1_gd_list)
            print("Avereage generational distance of algorithm 1:", algo1_avg_gd)
            print('Smallest generational distance of algorithm 1:', min(self.res1_gd_list))
            algo1_gd_std = np.std(np.array(self.res1_gd_list))
            print('Standard deviation of generational distance of algorithm 1:', algo1_gd_std,"\n")          
            if None in self.res2_list:
                tester.algorithm1_win_convergence += 1
        
        if None in self.res2_gd_list:
            print("No feasible solution found by algorithm 2")
        else:
            algo2_avg_gd = sum(self.res2_gd_list) / len(self.res2_gd_list)
            print("Avereage generational distance of algorithm 2:", algo2_avg_gd)
            print('Smallest generational distance of algorithm 2:', min(self.res2_gd_list))
            algo2_gd_std = np.std(np.array(self.res2_gd_list))
            print('Standard deviation of generational distance of algorithm 2:', algo2_gd_std,"\n")
            if None in self.res1_gd_list:
                tester.algorithm2_win_convergence += 1

        if not (None in self.res1_gd_list or None in self.res2_gd_list):
            if algo1_avg_gd < algo2_avg_gd:
                print("Algorithm 1 has a better approximation to the global optimum on average\n")
                tester.algorithm1_win_convergence += 1
            else:
                print("Algorithm 2 has a better approximation to the global optimum on average\n")
                tester.algorithm2_win_convergence += 1

    def visualize(self,res1,res2):
        opt1 = np.array([e.opt[0].F for e in res1.history])
        opt2 = np.array([e.opt[0].F for e in res2.history])
        pf = self.problem.pareto_front()
        n_eval1 = np.arange(len(res1.history))
        n_eval2 = np.arange(len(res2.history))
        plt.plot(n_eval1, opt1, label="Algorithm 1")
        plt.plot(n_eval2, opt2, label="Algorithm 2")
        plt.hlines(y=pf[0], xmin=0, xmax=max(n_eval1[-1],n_eval2[-1]), colors='purple', linestyles='--', lw=2)
        plt.title("Convergence Comparision")
        plt.xlabel("Genernation")
        plt.ylabel("Optimum in each generation")
        if pf[0] >= 0:
            plt.yscale("log")
        plt.legend()
        plt.show()
    
    def reset_count():
        tester.total_time_comparison = 0
        tester.total_convergence_comparison = 0
        tester.algorithm1_win_execution_time = 0
        tester.algorithm1_win_convergence = 0

In [3]:
algorithm1 = PSO()
algorithm2 = GA()
n_var = 20

In [4]:
ackley_tester = tester(algorithm1,algorithm2,get_problem("ackley", n_var=n_var, a=20, b=1/5, c=2 * np.pi))
ackley_tester.execute()
#ackley_tester.visualize()

Average execution time of algorithm 1: 3.516765904426575
Average execution time of algorithm 2: 6.518073296546936 

Shortest execution time of algorithm 1: 0.8055262565612793
Shortest execution time of algorithm 2: 4.674077272415161 

Standard deviation of execution time of algorithm 1: 1.219018635325539
Standard deviation of execution time of algorithm 2: 1.294859286771455 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.002450681111394193
Smallest generational distance of algorithm 1: 3.8625547205128896e-11
Standard deviation of generational distance of algorithm 1: 0.007352042388805332 

Avereage generational distance of algorithm 2: 0.023485849929668224
Smallest generational distance of algorithm 2: 0.015702737194818628
Standard deviation of generational distance of algorithm 2: 0.005063124906111496 

Algorithm 1 has a better approximation to the global optimum on average



In [5]:
griewank_tester = tester(algorithm1,algorithm2,get_problem("griewank", n_var=n_var))
griewank_tester.execute()
#griewank_tester.visualize()

Average execution time of algorithm 1: 1.4328692197799682
Average execution time of algorithm 2: 6.851402306556702 

Shortest execution time of algorithm 1: 0.6346213817596436
Shortest execution time of algorithm 2: 4.572067975997925 

Standard deviation of execution time of algorithm 1: 0.8687082835072449
Standard deviation of execution time of algorithm 2: 1.5861228877138696 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.23367893959177746
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 0.332257800339647 

Avereage generational distance of algorithm 2: 0.02778866028904551
Smallest generational distance of algorithm 2: 0.004815624556890441
Standard deviation of generational distance of algorithm 2: 0.015464593033517895 

Algorithm 2 has a better approximation to the global optimum on average



In [6]:
rastrigin_tester = tester(algorithm1,algorithm2,get_problem("rastrigin", n_var=n_var))
rastrigin_tester.execute()
#rastrigin_tester.visualize()

Average execution time of algorithm 1: 1.2937275886535644
Average execution time of algorithm 2: 6.729350757598877 

Shortest execution time of algorithm 1: 0.4212770462036133
Shortest execution time of algorithm 2: 4.85626745223999 

Standard deviation of execution time of algorithm 1: 1.3515682901537227
Standard deviation of execution time of algorithm 2: 1.8370852140120173 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 10.085512040407488
Smallest generational distance of algorithm 1: 0.9949592756015022
Standard deviation of generational distance of algorithm 1: 6.145119650498541 

Avereage generational distance of algorithm 2: 0.0028129929554467025
Smallest generational distance of algorithm 2: 0.0014046170907704436
Standard deviation of generational distance of algorithm 2: 0.0011953982288857212 

Algorithm 2 has a better approximation to the global optimum on average



In [39]:
rosenbrock_tester = tester(algorithm1,algorithm2,get_problem("rosenbrock", n_var=n_var))
rosenbrock_tester.execute()


Average execution time of algorithm 1: 1.4593555450439453
Average execution time of algorithm 2: 55.74319000244141 

Shortest execution time of algorithm 1: 0.4886946678161621
Shortest execution time of algorithm 2: 9.33597207069397 

Standard deviation of execution time of algorithm 1: 0.6745529889432655
Standard deviation of execution time of algorithm 2: 130.4290035654955 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 16.061136483156076
Smallest generational distance of algorithm 1: 9.432778620374856
Standard deviation of generational distance of algorithm 1: 2.5849653851521444 

Avereage generational distance of algorithm 2: 14.885868143448928
Smallest generational distance of algorithm 2: 0.18625171880743188
Standard deviation of generational distance of algorithm 2: 4.927159402630447 

Algorithm 2 has a better approximation to the global optimum on average



In [8]:
zakharov_tester = tester(algorithm1,algorithm2,get_problem("zakharov", n_var=n_var))
zakharov_tester.execute()

Average execution time of algorithm 1: 1.1449408054351806
Average execution time of algorithm 2: 24.48371069431305 

Shortest execution time of algorithm 1: 0.19353461265563965
Shortest execution time of algorithm 2: 0.4206254482269287 

Standard deviation of execution time of algorithm 1: 0.7526365451451487
Standard deviation of execution time of algorithm 2: 32.8446341670272 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.004156040223900548
Smallest generational distance of algorithm 1: 6.125155375838453e-06
Standard deviation of generational distance of algorithm 1: 0.00893198662193912 

Avereage generational distance of algorithm 2: 1.934818369454462
Smallest generational distance of algorithm 2: 0.4675457433472518
Standard deviation of generational distance of algorithm 2: 1.046036747088875 

Algorithm 1 has a better approximation to the global optimum on average



In [20]:
g01_tester = tester(algorithm1,algorithm2,get_problem("g1"))
g01_tester.execute()

Average execution time of algorithm 1: 2.4228418350219725
Average execution time of algorithm 2: 2.750700521469116 

Shortest execution time of algorithm 1: 1.8715553283691406
Shortest execution time of algorithm 2: 2.002570867538452 

Standard deviation of execution time of algorithm 1: 0.35107128625033224
Standard deviation of execution time of algorithm 2: 0.6494623019248142 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 1.6328125
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 1.1073099177313692 

Avereage generational distance of algorithm 2: 0.7185409892744946
Smallest generational distance of algorithm 2: 0.0006182285939004828
Standard deviation of generational distance of algorithm 2: 1.0955781466087653 

Algorithm 2 has a better approximation to the global optimum on average



In [10]:
g02_tester = tester(algorithm1,algorithm2,get_problem("g2"))
g02_tester.execute()

Average execution time of algorithm 1: 1.1755805492401123
Average execution time of algorithm 2: 24.721462535858155 

Shortest execution time of algorithm 1: 0.7087502479553223
Shortest execution time of algorithm 2: 2.5750064849853516 

Standard deviation of execution time of algorithm 1: 0.4393951200243119
Standard deviation of execution time of algorithm 2: 62.764378247648715 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.3822517509428537
Smallest generational distance of algorithm 1: 0.2585376308028928
Standard deviation of generational distance of algorithm 1: 0.06860833261830072 

Avereage generational distance of algorithm 2: 0.046183436533672295
Smallest generational distance of algorithm 2: 0.01615944683827686
Standard deviation of generational distance of algorithm 2: 0.02544375290555106 

Algorithm 2 has a better approximation to the global optimum on average



In [11]:
g03_tester = tester(algorithm1,algorithm2,get_problem("g3"))
g03_tester.execute()

Average execution time of algorithm 1: 0.22799651622772216
Average execution time of algorithm 2: 1.2367735385894776 

Shortest execution time of algorithm 1: 0.10557794570922852
Shortest execution time of algorithm 2: 0.38526225090026855 

Standard deviation of execution time of algorithm 1: 0.07467251163697966
Standard deviation of execution time of algorithm 2: 1.1488271602307618 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.6813953379960478
Smallest generational distance of algorithm 1: 0.3317180916601945
Standard deviation of generational distance of algorithm 1: 0.22339039263990637 

Avereage generational distance of algorithm 2: 0.8310796324287146
Smallest generational distance of algorithm 2: 0.4355922132760106
Standard deviation of generational distance of algorithm 2: 0.16197823385970975 

Algorithm 1 has a better approximation to the global optimum on average



In [12]:
g04_tester = tester(algorithm1,algorithm2,get_problem("g4"))
g04_tester.execute()

Average execution time of algorithm 1: 1.9795821189880372
Average execution time of algorithm 2: 2.294120216369629 

Shortest execution time of algorithm 1: 0.7595062255859375
Shortest execution time of algorithm 2: 1.5501809120178223 

Standard deviation of execution time of algorithm 1: 0.8794335211424706
Standard deviation of execution time of algorithm 2: 0.5609648044128299 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 1.1767406249418854e-08
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 3.366166393232793e-08 

Avereage generational distance of algorithm 2: 6.623707827432009
Smallest generational distance of algorithm 2: 1.8393792409733578
Standard deviation of generational distance of algorithm 2: 5.172000543257738 

Algorithm 1 has a better approximation to the global optimum on average



In [13]:
g05_tester = tester(algorithm1,algorithm2,get_problem("g5"))
g05_tester.execute()

Average execution time of algorithm 1: 1.1888376474380493
Average execution time of algorithm 2: 7.194760322570801 

Shortest execution time of algorithm 1: 0.7289042472839355
Shortest execution time of algorithm 2: 3.4468305110931396 

Standard deviation of execution time of algorithm 1: 0.23816274862269765
Standard deviation of execution time of algorithm 2: 3.4585025894943833 

Algorithm 1 has shorter average execution time

No feasible solution found by algorithm 1
No feasible solution found by algorithm 2


In [14]:
g06_tester = tester(algorithm1,algorithm2,get_problem("g6"))
g06_tester.execute()

Average execution time of algorithm 1: 0.8172484159469604
Average execution time of algorithm 2: 8.594055199623108 

Shortest execution time of algorithm 1: 0.29013967514038086
Shortest execution time of algorithm 2: 0.32143235206604004 

Standard deviation of execution time of algorithm 1: 0.5079825741471855
Standard deviation of execution time of algorithm 2: 5.6262863677895805 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 8.144522198563208e-05
Smallest generational distance of algorithm 1: 6.6283455453231e-06
Standard deviation of generational distance of algorithm 1: 5.9166581527269416e-05 

Avereage generational distance of algorithm 2: 68.12099702096802
Smallest generational distance of algorithm 2: 11.144215520668695
Standard deviation of generational distance of algorithm 2: 45.11648693213286 

Algorithm 1 has a better approximation to the global optimum on average



In [15]:
g07_tester = tester(algorithm1,algorithm2,get_problem("g7"))
g07_tester.execute()

Average execution time of algorithm 1: 1.6937083005905151
Average execution time of algorithm 2: 3.3995049238204955 

Shortest execution time of algorithm 1: 0.8015167713165283
Shortest execution time of algorithm 2: 2.6128644943237305 

Standard deviation of execution time of algorithm 1: 0.7863864236597956
Standard deviation of execution time of algorithm 2: 0.49997015633495656 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 1.6195788318522026
Smallest generational distance of algorithm 1: 0.19477344050018885
Standard deviation of generational distance of algorithm 1: 1.1061958567143713 

Avereage generational distance of algorithm 2: 4.393004408234178
Smallest generational distance of algorithm 2: 0.13238310311135137
Standard deviation of generational distance of algorithm 2: 4.688525718363379 

Algorithm 1 has a better approximation to the global optimum on average



In [19]:
g08_tester = tester(algorithm1,algorithm2,get_problem("g8"))
g08_tester.execute()

Average execution time of algorithm 1: 0.2744955062866211
Average execution time of algorithm 2: 0.9821463584899902 

Shortest execution time of algorithm 1: 0.18529391288757324
Shortest execution time of algorithm 2: 0.6235177516937256 

Standard deviation of execution time of algorithm 1: 0.1079031627360842
Standard deviation of execution time of algorithm 2: 0.20040055454793085 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.0
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 0.0 

Avereage generational distance of algorithm 2: 2.031717749317874e-08
Smallest generational distance of algorithm 2: 8.090750291955828e-15
Standard deviation of generational distance of algorithm 2: 5.0177513679734465e-08 

Algorithm 1 has a better approximation to the global optimum on average



In [17]:
g09_tester = tester(algorithm1,algorithm2,get_problem("g9"))
g09_tester.execute()

Average execution time of algorithm 1: 1.1148066997528077
Average execution time of algorithm 2: 3.00740909576416 

Shortest execution time of algorithm 1: 0.6135697364807129
Shortest execution time of algorithm 2: 2.036874771118164 

Standard deviation of execution time of algorithm 1: 0.41525142208278043
Standard deviation of execution time of algorithm 2: 0.7768074492174499 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.13967687785959698
Smallest generational distance of algorithm 1: 0.031653979628231355
Standard deviation of generational distance of algorithm 1: 0.0690352047256087 

Avereage generational distance of algorithm 2: 0.6728992999855109
Smallest generational distance of algorithm 2: 0.20247815732182062
Standard deviation of generational distance of algorithm 2: 0.3611122434838477 

Algorithm 1 has a better approximation to the global optimum on average



14
13
1
13
8
5


In [38]:
for x in zakharov_tester.res2_list:
    print(x.CV)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


TypeError: optimize_acquisition_function() takes 1 positional argument but 2 were given